# PR-22e-09 - Frequency Study print 3d
### $G  = 2.09\,G_0$
### $\tau_i = \{0.80, 0.63, 0.33, 0.25, 0.01, 0.00\}$

In [1]:
# initialize stuff
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import importlib
import logging

import numpy as np
import matplotlib.pyplot as plt

sys.path.append("/Users/oliver/Documents/p5control-bluefors-evaluation")
sys.path.append("/home/oliver/Documents/p5control-bluefors-evaluation")

from utilities.baseclass import BaseClass
from utilities.baseevaluation import BaseEvaluation
from utilities.baseplot import BasePlot
from utilities.ivevaluation import IVEvaluation

importlib.reload(sys.modules["utilities.baseclass"])
importlib.reload(sys.modules["utilities.baseevaluation"])
importlib.reload(sys.modules["utilities.baseplot"])
importlib.reload(sys.modules["utilities.ivevaluation"])

import utilities.basefunctions
importlib.reload(sys.modules["utilities.basefunctions"])

from utilities.corporate_design_colors_v4 import cmap
importlib.reload(sys.modules["utilities.corporate_design_colors_v4"])

importlib.reload(logging)
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format="%(message)s")
# pip install numpy matplotlib scipy tqdm torch hdf5view h5py PySide6

In [2]:
# Evaluation for unbroken
from utilities.ivevaluation import IVEvaluation
importlib.reload(sys.modules["utilities.ivevaluation"])

eva = IVEvaluation()
eva.file_directory = '/Volumes/speedyboy/measurement data 2/'
eva.file_folder = '2023-11-03_1_G0/'
eva.file_name = '2023-11-03_G0_stripline.hdf5'
eva.setAmplifications(10000, 1000)
eva.setV(.7e-3, voltage_bins=400)
eva.setI(100e-9, current_bins=1000)
eva.showMeasurements()

eva.downsample_frequency = 43


# Frequency Study (0.1V)
eva.title = "Frequency Study (0.1V)"
print(eva.title)
eva.sub_folder = "frequency study"
eva.setMeasurement("frequency_at_-20dBm")
eva.showKeys()
eva.setKeys(index_0=3, index_1=-3, norm=1e9)
eva.eva_temperature = False
eva.up_sweep, = eva.getMaps([1])
eva.saveData()

(base) ... BaseClass initialized.
(base eva) ... BaseEvaluation initialized.
(iv eva) ... IVEvaluation initialized.
(base eva) Available measurements:
critical field
frequency_at_-20dBm
frequency_at_-25dBm
frequency_at_15GHz
frequency_at_19.3GHz
frequency_at_7.8GHz
Frequency Study (0.1V)
(base eva) Measurement keys preview: ['nu=0.000GHz', 'nu=0.100GHz', 'nu=9.800GHz', 'nu=9.900GHz']


Frequency Study (0.1V)


100%|██████████| 201/201 [00:05<00:00, 37.40it/s]
/Users/oliver/Documents/p5control-bluefors-evaluation/utilities/ivevaluation.py:832: RuntimeWarning: Mean of empty slice
  dictionary["temperature"] = np.nanmean(
/Users/oliver/Documents/p5control-bluefors-evaluation/utilities/ivevaluation.py:843: RuntimeWarning: Mean of empty slice
  dictionary["temperature"] = np.nanmean(
(base) saveData()


In [3]:
# Execute All IVPlotting
%matplotlib qt
from utilities.ivplot import IVPlot
importlib.reload(sys.modules["utilities.ivplot"])

i = 0
eva = IVPlot()
eva.title = "Frequency Study (0.1V)"
eva.sub_folder = "frequency study"
eva.loadData()
eva.to_plot = eva.up_sweep
eva.title_of_plot = "Up Sweep"
eva.y_characters = [r"$\nu_\mathrm{AC}$", "Hz"]
eva.smoothing = True
eva.window_length = 5
eva.plot_T = False
eva.plot_all()

(base) ... BaseClass initialized.
(base eva) ... BaseEvaluation initialized.
(iv eva) ... IVEvaluation initialized.
(base) ... BaseClass initialized.
(base plot) ... BasePlot initialized.
(iv plot) ... IVPlot initialized.
Frequency Study (0.1V)
(base) loadData()
(iv plot) smoothing = True
(iv plot) window_length = 5
(base plot) saveFigure()
(base plot) saveFigure()


In [6]:
y_lim = (50, None)

Z, x, y = eva.up_sweep["differential_conductance"], eva.mapped["voltage_axis"], eva.mapped["y_axis"]

Z = Z[y_lim[0]:y_lim[1], :]
y= y[y_lim[0]:y_lim[1]]
x = x[:]
X, Y = np.meshgrid(x, y)
Z_base = np.ones(np.shape(Z))*-.05
factor = 1

Z -= np.nanmin(Z)
X /= np.nanmax(X) * len(x) / len(y) * factor
Y /= np.nanmax(Y) * factor
Z_base /= np.nanmax(Z) * factor
Z /= np.nanmax(Z) * factor

len(y)

151

In [7]:

from matplotlib import cbook, cm
from matplotlib.colors import LightSource

# Set up plot
fig, ax = plt.subplots(subplot_kw=dict(projection='3d'))

ls = LightSource(270, 45)
rgb = ls.shade(Z, cmap=cmap(), vert_exag=0.1, blend_mode='soft')

surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1,facecolors=rgb,
                       linewidth=0, antialiased=False, shade=False)
ax.invert_yaxis()

# ax.contourf(X, Y, Z, zdir='z', offset=-.1, cmap='viridis')

# ax.set_zlim(None, None)
# ax.contourf(X, Y, Z, zdir='x', offset=x[0], cmap=cmap())
# ax.contourf(X, Y, Z, zdir='y', offset=y[-1], cmap=cmap())

plt.show()

In [8]:
# Create triangular mesh for STL
 
# Original surface
vertices_top = np.column_stack((X.ravel(), Y.ravel(), Z.ravel()))

# Base surface (same X/Y, Z=0)
vertices_bottom = np.column_stack((X.ravel(), Y.ravel(), Z_base.ravel()))

# Combine both
vertices = np.vstack((vertices_top, vertices_bottom))
n = X.shape[0] * X.shape[1]  # number of vertices in each layer

ncols = len(x)
nrows = len(y)

faces = []

def idx(layer, i, j):
    return layer * n + i * ncols + j

for i in range(nrows - 1):
    for j in range(ncols - 1):
        # top surface
        faces.append([idx(0, i, j), idx(0, i+1, j), idx(0, i, j+1)])
        faces.append([idx(0, i+1, j), idx(0, i+1, j+1), idx(0, i, j+1)])
        
        # bottom surface (inverted winding order)
        faces.append([idx(1, i, j), idx(1, i, j+1), idx(1, i+1, j)])
        faces.append([idx(1, i+1, j), idx(1, i, j+1), idx(1, i+1, j+1)])
        
        # sides (walls)
        faces.append([idx(0, i, j), idx(0, i+1, j), idx(1, i+1, j)])
        faces.append([idx(0, i, j), idx(1, i+1, j), idx(1, i, j)])
        
        faces.append([idx(0, i+1, j), idx(0, i+1, j+1), idx(1, i+1, j+1)])
        faces.append([idx(0, i+1, j), idx(1, i+1, j+1), idx(1, i+1, j)])
        
        faces.append([idx(0, i+1, j+1), idx(0, i, j+1), idx(1, i, j+1)])
        faces.append([idx(0, i+1, j+1), idx(1, i, j+1), idx(1, i+1, j+1)])
        
        faces.append([idx(0, i, j+1), idx(0, i, j), idx(1, i, j)])
        faces.append([idx(0, i, j+1), idx(1, i, j), idx(1, i, j+1)])

import numpy
from stl import mesh
faces = np.array(faces)

m = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
for i, f in enumerate(faces):
    for j in range(3):
        m.vectors[i][j] = vertices[f[j], :]

# Save STL file
m.save(eva.title+".stl")

In [9]:
# Evaluation for unbroken
from utilities.ivevaluation import IVEvaluation
importlib.reload(sys.modules["utilities.ivevaluation"])

eva = IVEvaluation()
eva.file_directory = '/Users/oliver/Documents/measurement data'
eva.file_folder = '23 10 PR-22e-09/'
eva.file_name = '2023-11-03_G0_antenna.hdf5'
eva.setAmplifications(10000, 1000)
eva.setV(.7e-3, voltage_bins=350)
eva.setI(100e-9, current_bins=1000)
eva.showMeasurements()

eva.downsample_frequency = 43


# Amplitude Study (19.3GHz)
eva.title = "Amplitude Study (19.3GHz)"
print(eva.title)
eva.sub_folder = "amplitude study"
eva.setMeasurement("frequency_at_19.3GHz")
eva.showKeys()
eva.setKeys(index_0=3, index_1=-3, norm=1)
eva.eva_temperature = False
eva.up_sweep, = eva.getMaps([1])
eva.up_sweep, = eva.getMapsAmplitude([eva.up_sweep])
eva.y_axis = eva.amplitude_axis
eva.saveData()



(base) ... BaseClass initialized.
(base eva) ... BaseEvaluation initialized.
(iv eva) ... IVEvaluation initialized.


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/Users/oliver/Documents/measurement data23 10 PR-22e-09/2023-11-03_G0_antenna.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [21]:
# Execute All IVPlotting
%matplotlib qt
from utilities.ivplot import IVPlot
importlib.reload(sys.modules["utilities.ivplot"])

eva = IVPlot()
eva.title = "Amplitude Study (19.3GHz)"
eva.sub_folder = "amplitude study"
eva.loadData()
eva.to_plot = eva.up_sweep
eva.title_of_plot = "Up Sweep"
eva.y_characters = [r"$\nu_\mathrm{AC}$", "Hz"]
eva.smoothing = True
eva.window_length = 5
eva.plot_T = False
eva.plot_all()

(base) ... BaseClass initialized.
(base eva) ... BaseEvaluation initialized.
(iv eva) ... IVEvaluation initialized.
(base) ... BaseClass initialized.
(base plot) ... BasePlot initialized.
(iv plot) ... IVPlot initialized.
Amplitude Study (19.3GHz)
(base) loadData()
(iv plot) smoothing = True
(iv plot) window_length = 5
(base plot) saveFigure()
(base plot) saveFigure()
